In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm ##### pip install tqdm ได้เลย
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from joblib import load ##### น่าจะเป็น python build-in นะไม่แน่ใจ

RandomState = 1

### Setting up for exploration

- declare function

In [ ]:
def plotHistrogram(df:pd.DataFrame, hue=None, palette=None):
    cols = df.columns
    plot_cols = 5
    plot_row = round((len(cols)/plot_cols)+1)
    plot_num = 1
    plt.figure(figsize=(plot_cols*6, plot_row*6))
    for i in tqdm(cols):
        ax = plt.subplot(plot_row, plot_cols, plot_num)
        sns.histplot(data=df, x=i, kde=True, hue=hue, palette=palette)
        plot_num += 1
    plt.show()

def plotScatter(df:pd.DataFrame, hue=None):
    cols = df.columns
    plot_cols = len(cols)
    plot_row = len(cols)
    plot_num = 1
    plt.figure(figsize=(plot_cols*8, plot_row*8))
    for i in tqdm(cols):
        for j in cols:
            ax = plt.subplot(plot_row, plot_cols, plot_num)
            sns.scatterplot(data=df, x=i, y=j, hue=hue)
            plot_num += 1
    plt.show()

def removeNumericalOutlier(df:pd.DataFrame, feature_list:list):
    new_df = df.copy()
    for feature in feature_list:       
        q1 = df[feature].quantile(0.25)
        q3 = df[feature].quantile(0.75)
        IQR = q3 - q1
        lower_bound = q1 - 1.5*IQR
        upper_bound = q3 + 1.5*IQR
        new_df = new_df[(new_df[feature]>lower_bound)&(new_df[feature]<upper_bound)]
    return new_df

def plotClusteringScore(res_kmean):
    kmean_num = list(res_kmean.keys())
    kmean_WCSS = [res_kmean[i]['WCSS'] for i in kmean_num]
    kmean_sil_score = [res_kmean[i]['silhouette_score'] for i in kmean_num]
    
    fig, ax = plt.subplots(ncols=2, figsize=(14, 3))
    ax[0].scatter(kmean_num, kmean_WCSS)
    ax[0].plot(kmean_num, kmean_WCSS)
    ax[0].set_xlabel("K")
    ax[0].set_ylabel("WCSS")
    
    ax[1].scatter(kmean_num, kmean_sil_score)
    ax[1].plot(kmean_num, kmean_sil_score)
    ax[1].set_xlabel("K")
    ax[1].set_ylabel("silhouette_score")
    
    
    plt.legend(loc='upper right')
    plt.show()


### Data Exploration

##### explore match data

> ไฟลนี้มาจากการที่เอาไฟล match กับ player โดยในไฟล player จะมีข้อมูลของผู้เล่นทุกคนใน match id นั้นๆ ซึ่งนำข้อมูลของแต่ละคนมาบวกกันโดยแบ่งเป็นทีม radiant กับ dire

> หมายความว่าในไฟลนี้จะเป็นไฟลที่บ่งบอกถึงในแต่ละ match แต่ละทีมมี statistic อย่างไร

In [ ]:
new_match_df = pd.read_csv("./new_dataframe/new_match_df.csv", index_col=0)
new_match_df['radiant_total_leaver_status'] = new_match_df['radiant_total_leaver_status'].replace(list(range(10)), [0, 0, 1, 1, 1, 1, 1, 1, 1, 1])
new_match_df['dire_total_leaver_status'] = new_match_df['dire_total_leaver_status'].replace(list(range(10)), [0, 0, 1, 1, 1, 1, 1, 1, 1, 1])
new_match_df.head()

- histrogram plot

> the duration of gameplay is 2476 seconds and the standard deviation is 634.631261 and <font color=red>**the data also have outliers**</font> if we remove the outliers of this feature will affect the outliers of other features

> 36.7% of first blood time is in range 0 - 41.55

> tower and barracks status obviously show the impact on wins and losses because it is measured at the last tower, which to get it must eliminate the previous tower first.

> The winning team obviously shows more gold spent, GPM, XPM, kills, deaths, assists, level and damage to tower and heroes

> **Surprising stuns duration and leaver status do not impact wins and losses**

> Most games end at 2476 seconds, the average level of the whole team is 18, and the killing score of winner team is 44.7 and 29.37 for the loser team


In [ ]:
rm_outliers_match_df = removeNumericalOutlier(new_match_df, ['duration'])
plotHistrogram(rm_outliers_match_df, hue='radiant_win')

- scatter plot

> obviously see better sepration except stuns, LH, denies and assist

<font color=red>**ไม่แนะนำให้รันโค้ดด้านล่างนี้ ใช้เวลาอย่างต่ำ 20 นาที**</font><br>

ดูภาพนี้เอาละกัน

<center><img src="./new_dataframe/match_scatter.png"><center>

In [ ]:
# plotScatter(rm_outliers_match_df.drop(['match_id', 'tower_status_radiant', 
#                                         'tower_status_dire', 'barracks_status_dire', 
#                                         'barracks_status_radiant'], axis=1), hue='radiant_win')

- does leaver status shown anything?

> <u>**ANS**</u> the data of abandoned matches is too less

In [ ]:
plotHistrogram(rm_outliers_match_df.drop(['match_id'], axis=1), 'radiant_total_leaver_status')

- cluster radiant and dire total stuns
    - ['radiant_total_stuns', 'dire_total_stuns']

> The cluster of stuns duration doesn't indicate any significant terms. But the clusters with high stuns duration will slightly increase gameplay time and it also affects the killing score

> เราเห็นจากการทำ histogram แล้วเราประหลาดใจที่ว่าตัว stun duration ไม่ได้ส่งผลอะไรกับอัตราการแพ้ชนะเลย แล้วมันส่งผลกับอะไรกันแน่ เลยมาลอง cluster ดูโดย

> assumtion ที่ว่ามันน่าจะส่งผลต่ออัตราการชนะเพิ่มขึ้นมา

> result มันไม่ได้ส่งผลกับอัตราแพ้ชนะเลย มันส่งผลกับระยะเวลาใน match นั้นๆอย่างเดียวแต่ก็ไม่ strong พอที่จะ judge มัน

In [ ]:
cluster_stuns = load('cluster_stuns.joblib')
new_match_df['cluster_stuns'] = cluster_stuns.labels_
rm_outliers_match_df = removeNumericalOutlier(new_match_df, ['duration'])
plotHistrogram(rm_outliers_match_df, hue='cluster_stuns')

##### explore teamfight data

> ข้อมูลในส่วนนี้อธิบายถึง team fight ทั้งหมดในแต่ละ match โดย feature ของมันมาจาก sum value ของผู้เล่น และเวลาที่เกิด team fight โดยที่เราเอาเวลานั้นไปหาเวลาที่ใกล้ที่สุดที่หารด้วย 60 ลงตัว เนื่องจากในไฟลหนึ่งของ dataset เก็บค่า gold xp ต่างๆเอาไว้โดย dependent on times every 60 seconds 

In [ ]:
team_stat_on_time = pd.read_csv("./new_dataframe/team_stat_on_time.csv", index_col=0)
new_teamfight_df = pd.read_csv("./new_dataframe/new_teamfight_df.csv", index_col=0)
##### calculate time before team fight and merge to team status on times
new_teamfight_df['time_before_fight'] = new_teamfight_df['start'] - new_teamfight_df['start']%60
new_teamfight_df = new_teamfight_df.drop(['start', 'end'], axis=1)
teamfight = new_teamfight_df.merge(team_stat_on_time, left_on=['match_id', 'time_before_fight'], right_on=['match_id', 'times'])
teamfight = teamfight.drop(['times'], axis=1)
teamfight['radiant_benefit'] = (teamfight['radiant_total_delta_gold'] + teamfight['radiant_total_delta_xp']) > (teamfight['dire_total_delta_gold'] + teamfight['dire_total_delta_xp'])
teamfight['radiant_benefit'] = teamfight['radiant_benefit'].replace([False, True], [0, 1])
teamfight.head()

- histogram plot

> We don't consider delta gold and XP because the benefit of a team fight calculate from delta gold and XP after the team fight occurs

> The histogram plot didn't show any feature that shows the team fight benefit but <font color=yellow>amazing that the gold and XP each team before the team fight doesn't affect to the benefit of that team fight</font>

> <font color=red>**Time before fight data have outliers**</font> if we remove the outliers of this feature will affect the outliers of other features


In [ ]:
rm_outliers_teamfight_df = removeNumericalOutlier(teamfight, ['time_before_fight'])
plotHistrogram(rm_outliers_teamfight_df.drop(['match_id'], axis=1), hue='radiant_benefit')

- scatter plot

> the result is also the same as the histogram plot it doesn't have any feature that seems to impact on team fight benefit

<font color=red>**ไม่แนะนำให้รันโค้ดด้านล่างนี้ ใช้เวลาอย่างต่ำ 20 นาที**</font>

<center><img src="./new_dataframe/teamfight_status_scatter.png"><center>


In [ ]:
# plotScatter(rm_outliers_teamfight_df.drop(['match_id'], axis=1), hue='radiant_benefit')

- cluster delta gold and XP
    - ['radiant_total_delta_gold', 'dire_total_delta_gold', 'radiant_total_delta_xp', 'dire_total_delta_xp']

> Cluster 1 is show the benefit in teamfight mean is 1036 and the team fight time in range 0 to 3720 with mean 1065

> Cluster 0 & 2 is show the benefit in teamfight that more higher and lower than cluster 1 and the team fight time in range 420 to 3720 with mean 1966

> Those cluster show the team gold, XP, LH score are dependent on time before fight and it also effect to benefit in team fight but not the significant terms for predict team fight win and lose

> ข้อมูลพวก delta gold กับ XP มันเป็นข้อมูลที่เอามาใช้คิดตัว benefit in team fight ซึ่งเราอยากรู้ว่าถ้าเป็น raw data พวกนีแล้วจริงๆมันทำให้เกิด cluster ตัวไหนบ้าง

> assumption delta gold and XP น่าจะเป็นคลัสเตอร์ที่ทำให้เรารู้ได้ว่าอะไรกันแน่ที่ส่งผลต่อความได้เปรียบใน team fight

> result ทำให้เราได้รู้ว่าพวก delta gold and XP ทั้งหลายใน team fight มันเกิดจากเวลายิ่ง gameplay นานขึ้นผลต่างของ gold and XP ใน teamfight ยิ่งสูงขึ้น

In [ ]:
cluster_delta_stat = load('cluster_delta_stat.joblib')
teamfight['cluster_delta_stat'] = cluster_delta_stat.labels_
rm_outliers_teamfight_df = removeNumericalOutlier(teamfight, ['time_before_fight'])
plotHistrogram(rm_outliers_teamfight_df, hue='cluster_delta_stat', palette=['r', 'g', 'b'])

- cluster team stat
    - ['radiant_gold_t_', 'dire_gold_t_', 'radiant_xp_t_', 'dire_xp_t_', 'radiant_lh_t_', 'dire_lh_t_']

> The result is also the same as the delta gold and XP cluster it shows 2 clusters that are different on time before the team fight

> ส่วนคลัสเตอร์อันนี้ก็แค่อยากรู้ว่า gold xp lh stat พวกนี้ก่อนทีมไฟท์มันบอกอะไรได้บ้าง
> assumption ถ้ามันไม่ได้บอกถึงความได้เปรียบมันก็น่าจะบอกถึงเวลา เพราะของพวกนี้ยิ่งเวลานานๆมันก็น่าจะเพิ่มขึ้นเรื่อย

> result ตาม assumption เลย

In [ ]:
cluster_teamfight_stat = load('cluster_teamfight_stat_2.joblib')
teamfight['cluster_teamfight_stat2'] = cluster_teamfight_stat.labels_
rm_outliers_teamfight_df = removeNumericalOutlier(teamfight, ['time_before_fight'])
plotHistrogram(rm_outliers_teamfight_df, hue='cluster_teamfight_stat2')

##### explore player statistic in match

In [ ]:
player_df = pd.read_csv('players.csv')
player_df = player_df.iloc[:, 0:25]
player_df = player_df.drop(['gold', 'account_id', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5'], axis=1)
player_df['stuns'] = player_df['stuns'].replace(['None'], [0.0])
player_df['stuns'] = player_df['stuns'].astype(np.float64)
# player_df['leaver_status'] = player_df['leaver_status'].replace(list(range(5)), [0, 0, 1, 1, 1])
player_df.head()

- cluster player score
    - ['gold_spent', 'gold_per_min', 'xp_per_min', 'kills', 'deaths', 'assists', 'denies', 'last_hits', 'stuns', 'hero_damage', 'hero_healing', 'tower_damage']

> cluster 0 show the lower statistic and we guess it should be the support hero

> cluster 1 show the better score, damage and gold spent we guess that is the carry or farming hero type

> คลัสเตอร์ player score ใช้ฟีเจอร์ต่างๆที่เกี่ยวกับ hero ซึ่ง features พวกนี้น่าจะบ่งบอกถึงประเภทของฮีโร่

> assumption บ่งบอก role ของ hero ต่างๆ

> result บ่งบอกแค่ตัวที่ใช้เงินมากๆตัว carry หรือ farming

In [ ]:
player_stat_cluster = load('player_stat_cluster.joblib')
player_df['player_stat_cluster'] = player_stat_cluster.labels_
plotHistrogram(player_df, hue='player_stat_cluster')

In [ ]:
hero_name = pd.read_csv('hero_names.csv')
hero_name['hero_id'] = hero_name['hero_id']
hero_lookup_table = dict(zip(hero_name['hero_id'],hero_name['localized_name']))
hero_lookup_table[0] = 'missing'
player_df['hero_name'] = player_df['hero_id'].apply(lambda id : hero_lookup_table[id])
print(player_df.loc[player_df['player_stat_cluster']==1]['hero_name'].value_counts())
print(player_df.loc[player_df['player_stat_cluster']==0]['hero_name'].value_counts())

- answer the assumtion

> cluster 1 shows the carry hero and the top 5 heroes of this cluster is Shadow Fiend, Windranger, Alchemist, Juggernaut, Invoker

> cluster 0 show the other roles hero the top 5 heroes of this cluster is Windranger, Earthshaker, Slardar, Dazzle, Rubick